<a href="https://colab.research.google.com/github/hholl19/Deep-Leaning-Class-Notes/blob/hholl19_added_things/Applied_Deep_Learning_2nd_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
import pandas as pd
import numpy as np
import os
import pathlib 
from pathlib import Path
#import cv2
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dropout, Dense, Flatten, Conv2D, MaxPooling2D 
from keras.layers.normalization import BatchNormalization 
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split

from numpy import expand_dims
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from matplotlib import pyplot